<a href="https://colab.research.google.com/github/XiaoQLee/Python-100-Days/blob/master/step1_3_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 資料變數名稱

## cor
#### 全報酬率清單
## _years
##### 年份清單
## _[年份]
##### 起始年份往後5年之整理後月報酬
## _[年份]_cor
##### 期間的相關係數
## _[年份]_firms
##### 期間所有可使用之公司清單
## _[年份]_[firm]_f
##### 期間該公司前f大相關係數公司清單
## _[年份]_[firm]
##### 期間該公司前f大相關係數公司月報酬
##### 其中 [firm]_port 欄位是配對交易報酬
____________________以下年份為交易年份_________________
## beta
##### 公司與基準投組beta
## port
##### 前f相關公司之等權報酬
## retdiff
##### 交易期間每月retdiff
## fret
##### 交易期間所有公司報酬
## rf
##### 無風險報酬


In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import time

def _index(df): ##建立索引列
    df.index = df.date ##以date作為index
    df.drop('date', axis = 1, inplace = True)  ##刪除date欄位
    df = df/100 ##處理百分比
    return df
    
def _data(df):
    df = df.dropna(axis=1, how='any', inplace=False)  ##處離期間空值刪除公司
    return df

In [ ]:
url = '/Users/XiaoQ/py/data/'
cor = _index(pd.read_csv(url+'cor.csv'))    ##輸入、整理檔案

In [ ]:
years = [] #建立年份清單
for year in range(1990,2020):
    years += [str(year)] 
    
beta = pd.DataFrame(index=years[5:], columns = cor.columns)
retdiff = pd.DataFrame(index=cor.index[60:], columns = cor.columns)
port = pd.DataFrame(index=cor.index[60:], columns = cor.columns)
_lyears = len(years)-5


In [ ]:
def get_beta(f):
    
    _time = time.time()
    #擷取形成期
    t = 0 #自第0期開始 擷取5年的資料，將資料整理後指派給 "_[年份]"
    for t in range (_lyears):
    #for t in range (5):
        globals()['_%s'%(years[t])] = _data(cor.iloc[12*t:12*(t+6)]).iloc[:60]

        #計算相關係數
        #指派給"_[年份]_cor"
        globals()['_%s_cor'%(years[t])] = globals()['_%s'%(years[t])].corr()

        #取出期間公司列表
        globals()['_%s_firms'%(years[t])] = globals()['_%s_cor'%(years[t])].columns

        lf = len(globals()['_%s_firms'%(years[t])])
        for firm in range(lf):
            #該公司前50大相關
            #指派給"_[年份]_[firm]_f"
            globals()['_%s_%s_%s'%(years[t], globals()['_%s_firms'%(years[t])][firm],str(f))] = \
            globals()['_%s_cor'%(years[t])][globals()['_%s_firms'%(years[t])][firm]].sort_values(ascending=False).index[1:f+1]

            #設立月報酬表格
            #指派給"_[年份]_[firm]"
            globals()['_%s_%s'%(years[t], globals()['_%s_firms'%(years[t])][firm])] = \
            globals()['_%s'%(years[t])][globals()['_%s_%s_%s'%(years[t], globals()['_%s_firms'%(years[t])][firm],str(f))]]

            #計算投資組合報酬
            globals()['_%s_%s'%(years[t],globals()['_%s_firms'%(years[t])][firm])]['%s_port'%(globals()['_%s_firms'%(years[t])][firm])] = \
            globals()['_%s_%s'%(years[t],globals()['_%s_firms'%(years[t])][firm])].sum(axis = 1)/f

            #計算beta並寫入df_beta中
            beta.loc[years[t+5],globals()['_%s_firms'%(years[t])][firm]] = \
            sm.OLS\
            (globals()['_%s'%(years[t])][globals()['_%s_firms'%(years[t])][firm]],\
            sm.add_constant(globals()['_%s_%s'%(years[t], globals()['_%s_firms'%(years[t])][firm])]['%s_port'%(globals()['_%s_firms'%(years[t])][firm])]))\
            .fit().params[1]
    
    print ('beta 計算時間：%.4f s'%(time.time()-_time))

In [ ]:
port = pd.DataFrame(index=cor.index[60:], columns = cor.columns)
dates = cor.index


def form_port(f):
    _time = time.time()
    
    for t in range(_lyears): #t==0
        _lfirms = len(globals()['_%s_firms'%(years[t])]) #_lfirms==lens of _1990_firms 
        a=(t+5)*12
        b=(t+6)*12
        for date in range (a,b): #date== 60
            for firm in range(_lfirms): #firm == 1101
                port.at[dates[date],globals()['_%s_firms'%(years[t])][firm]]=0 #(1995-01,1101)=0
                for p in range(f): #(1995-01,1101)=port
                    port.at[dates[date],globals()['_%s_firms'%(years[t])][firm]] += \
                    cor.loc[dates[date]][globals()['_%s_%s_%s'%(years[t],globals()['_%s_firms'%(years[t])][firm],str(f))][p]]/f

    print ('基準投組運算時間：%.4f s'%(time.time()-_time))

In [ ]:
rf = _index(pd.read_csv(url+'mktrf.csv'))[['rf']].loc[port.index[0]:]/12
retdiff = pd.DataFrame(index=port.index, columns = port.columns)
fret = cor.iloc[60:]

In [ ]:
def _retdiff(f):
    _time = time.time()
    ly = len(retdiff.index)
    lx = len(retdiff.columns)
    
    for y in range(ly):
        _y = int(y/12)
        for x in range(lx):
            retdiff.iat[y,x] = beta.iat[_y,x]*(port.iat[y,x]-rf.iat[y,0])-(fret.iat[y,x]-rf.iat[y,0])
    print ('RetDiff 運算時間：%.4f s'%(time.time()-_time))

In [ ]:
if __name__ == '__main__':
    f = 15
    get_beta(f)
    form_port(f)
    _retdiff(f)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


beta 計算時間：153.7515 s
基準投組運算時間：703.2088 s
RetDiff 運算時間：32.6833 s


In [ ]:
False in port.columns==beta.columns

False

In [ ]:
len(_2000_1101_15)

15

In [ ]:
#retdiff.to_csv(url+'retdiff_15.csv')
port.to_csv(url+'port15.csv')

In [ ]:
def res_test(year,_t,firm,f):
    print('FFTTTT is Fine')
    print(False in (fret.index==rf.index) & (port.index==retdiff.index) & (port.index==rf.index))
    print(False in (fret.columns==beta.columns) & (port.columns==retdiff.columns) & (port.columns==beta.columns))
    print((len(fret.columns)==len(beta.columns)) & (len(port.columns)==len(retdiff.columns)) & (len(port.columns)==len(beta.columns)))
    print((len(fret.index)==len(beta.index)*12) & (len(port.index)==len(retdiff.index)) & (len(port.index)==len(rf.index)))
    print((retdiff.at[_t,'%s'%(firm)])==(beta.at['%s'%(year),'%s'%(firm)]*(port.at[_t,'%s'%(firm)]-rf.at[_t,'rf'])-(fret.at[_t,'%s'%(firm)]-rf.at[_t,'rf'])))
    print((port.at[_t,'%s'%(firm)])==(cor.loc[_t][globals()['_%s_%s_%s'%(year-5,firm,f)]].sum()/f))
    

In [ ]:
res_test(2015,'2015-09',2337,15)

FFTTTT is Fine
False
False
True
True
True
False
